In [1]:
import numpy as np
# for numerical calculations, linear algebra, multi-dimensional arrays, etc.
import pandas as pd
# for data manipulation, analysis; useful for structural data, etc.

In [2]:
%pip install your-package-name

Note: you may need to restart the kernel to use updated packages.


In [12]:
data = pd.read_csv('dataset.csv')

In [6]:
data.sample(5)

,Category,Message
2791,spam,U’ve Bin Awarded £50 to Play 4 Instant Cash. C...
3894,ham,Have you heard from this week?
1989,ham,"Sorry, I'll call later"
179,ham,Text her. If she doesnt reply let me know so i...
2292,ham,Remind me how to get there and I shall do so


In [8]:
data.shape

(5572, 2)

In [ ]:
data.info()

In [ ]:
data.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [ ]:
data.sample(5)

In [ ]:
data.rename(columns={'v1':'target','v2':'text'},inplace=True)
data.sample(5)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
data['target'] = encoder.fit_transform(data['target'])

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data=data.drop_duplicates(keep='first')

In [ ]:
data.duplicated().sum()

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.pie(data['target'].value_counts(), labels=['ham','spam'],autopct="%0.2f")

In [ ]:
%pip install nltk
import nltk
nltk.download('punkt')

In [ ]:
data['num_characters']=data['text'].apply(len)

In [ ]:
data.head()

In [ ]:
data['num_words']=data['text'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
data['num_sentences']=data['text'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
data.head()

In [ ]:
data[['num_characters','num_words','num_sentences']].describe()

In [ ]:
#not_spam/ham
data[data['target']==0][['num_characters','num_words','num_sentences']].describe()

In [ ]:
#spam
data[data['target']==1][['num_characters','num_words','num_sentences']].describe()

In [ ]:
import seaborn as sns

In [ ]:
sns.histplot(data[data['target']==0]['num_characters'])
sns.histplot(data[data['target']==1]['num_characters'],color='red')

In [ ]:
sns.histplot(data[data['target']==0]['num_words'])
sns.histplot(data[data['target']==1]['num_words'],color='red')

In [ ]:
sns.pairplot(data,hue='target')

In [ ]:
data_numeric = data.select_dtypes(include=["number"])
data_numeric.corr()

In [ ]:

# Step 1: Filter out non-numeric data
numeric_data = data.select_dtypes(include=[np.number])

# Step 2: Calculate the correlation matrix
correlation_matrix = numeric_data.corr()

# Step 3: Create the heatmap
sns.heatmap(correlation_matrix, annot=True)

# Display the heatmap
import matplotlib.pyplot as plt
plt.show()

In [ ]:
%pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
import string

In [ ]:
from nltk.stem import PorterStemmer

def transform_text(text):
  # Lowercase the text
  text = text.lower()

  # Tokenize the text
  text = nltk.word_tokenize(text)

  y = []

  # Filter out non-alphanumeric characters
  for i in text:
      if i.isalnum():
          y.append(i)

  # Remove stop words and punctuation
  text = y[:]
  y.clear()

  for i in text:
      if i not in stopwords.words('english') and i not in string.punctuation:
          y.append(i)

  # Stem the words
  text = y[:]
  y.clear()

  ps = PorterStemmer()  # Instantiate PorterStemmer

  for i in text:
      y.append(ps.stem(i))

  # Return the transformed text
  return " ".join(y)

In [ ]:
transform_text('I am gonna be home soon and i dont want to talk about this stuff anymore tonight, k? Ive cried enough today.')

In [ ]:
# Import the necessary module
from nltk.stem import PorterStemmer

# Define the ps variable
ps = PorterStemmer()

# Call the transform_text function with the input text
transformed_text = transform_text('I am gonna be home soon and i dont want to talk about this stuff anymore tonight, k? Ive cried enough today.')

# Print the transformed text
print(transformed_text)

In [ ]:
data['text'][10]

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
ps.stem('loving')

In [ ]:
data['transformed_text'] = data['text'].apply(transform_text)

In [ ]:
data.head()

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(width=500,height=500,min_font_size=10,background_color='white')

In [ ]:
spam_wc = wc.generate(data[data['target']==1]['transformed_text'].squeeze().str.cat(sep=" "))

In [ ]:
plt.imshow(spam_wc)

In [ ]:
ham_wc = wc.generate(data[data['target']==0]['transformed_text'].squeeze().str.cat(sep=" "))
plt.imshow(ham_wc)

In [ ]:
data.head()

In [ ]:
spam_corpus = []
for msg in data[data['target']==1]['transformed_text'].tolist():
  for word in msg.split():
    spam_corpus.append(word)

In [ ]:
len(spam_corpus)

In [ ]:
from collections import Counter
sns.barplot(pd.DataFrame(Counter(spam_corpus).most_common(30))[0],)

In [ ]:
sns.barplot(pd.DataFrame(Counter(spam_corpus).most_common(30))[1],)

In [ ]:
sns.barplot(x=pd.DataFrame(Counter(spam_corpus).most_common(30))[0], y=pd.DataFrame(Counter(spam_corpus).most_common(30))[1], hue=data['target'])
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
ham_corpus = []
for msg in data[data['target']==0]['transformed_text'].tolist():
  for word in msg.split():
    ham_corpus.append(word)

In [ ]:
len(ham_corpus)

In [ ]:
sns.barplot(x=pd.DataFrame(Counter(ham_corpus).most_common(30))[0], y=pd.DataFrame(Counter(ham_corpus).most_common(30))[1], hue=data['target'])
plt.xticks(rotation='vertical')
plt.show()

MODEL BUILDING

In [ ]:
data.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features=3000)

In [ ]:
X = tfidf.fit_transform(data['transformed_text']).toarray()

In [ ]:
X.shape

In [ ]:
y = data['target'].values

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [ ]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
gnb.fit(X_train,y_train)
y_pred1 = gnb.predict(X_test)
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))

In [ ]:
mnb.fit(X_train,y_train)
y_pred2 = mnb.predict(X_test)
print(accuracy_score(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))

In [ ]:
bnb.fit(X_train,y_train)
y_pred3 = bnb.predict(X_test)
print(accuracy_score(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)

In [ ]:
clfs = {
    'SVC' : svc,
    'KN' : knc,
    'NB': mnb,
    'DT': dtc,
    'LR': lrc,
    'RF': rfc,
    'AdaBoost': abc,
    'BgC': bc,
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [ ]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)

    return accuracy,precision

In [ ]:
train_classifier(svc,X_train,y_train,X_test,y_test)

In [ ]:
accuracy_scores = []
precision_scores = []

for name,clf in clfs.items():

    current_accuracy,current_precision = train_classifier(clf, X_train,y_train,X_test,y_test)

    print("For ",name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)

    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)

In [ ]:
performance_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy':accuracy_scores,'Precision':precision_scores}).sort_values('Precision',ascending=False)

In [ ]:
performance_df

In [ ]:
performance_df1 = pd.melt(performance_df, id_vars = "Algorithm")

In [ ]:
performance_df1

In [ ]:
sns.catplot(x = 'Algorithm', y='value',
               hue = 'variable',data=performance_df1, kind='bar',height=5)
plt.ylim(0.5,1.0)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# model improve
# 1. Change the max_features parameter of TfIdf

In [ ]:
temp_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy_max_ft_3000':accuracy_scores,'Precision_max_ft_3000':precision_scores}).sort_values('Precision_max_ft_3000',ascending=False)

In [ ]:
temp_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy_scaling':accuracy_scores,'Precision_scaling':precision_scores}).sort_values('Precision_scaling',ascending=False)

In [ ]:
new_df = performance_df.merge(temp_df,on='Algorithm')

In [ ]:
new_df_scaled = new_df.merge(temp_df,on='Algorithm')

In [ ]:
temp_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy_num_chars':accuracy_scores,'Precision_num_chars':precision_scores}).sort_values('Precision_num_chars',ascending=False)

In [ ]:
new_df_scaled.merge(temp_df,on='Algorithm')